In [0]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
project_path = r'/content/gdrive/My Drive/Colab Notebooks/embedding_project/'
base_path = r'/content/gdrive/My Drive/Colab Notebooks/embedding_project/CSE5525_Project_Emoji_Prediction-master/data/'

In [0]:
train_file = project_path + 'updated_data/' + 'train_new.npy'
train_label = project_path + 'updated_data/' + 'train_label_new.npy'
test_file =  project_path + 'updated_data/' +  'test_new.npy'
test_label =  project_path + 'updated_data/' + 'test_label_new.npy'

In [0]:
X_train = np.load(train_file)
print(X_train.shape)
Y_train = np.load(train_label)
print(Y_train.shape)
X_test = np.load(test_file)
print(X_test.shape)
Y_test = np.load(test_label)
print(Y_test.shape)

(491664, 34)
(491664, 20)
(49999, 34)
(49999, 20)


In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, CuDNNLSTM, SpatialDropout1D, Activation, Dropout, Bidirectional, Conv1D, MaxPooling1D, BatchNormalization

vocab_size = 20827
EMBEDDING_SIZE = 100
HIDDEN_LAYER_SIZE = 40
BATCH_SIZE = 2048
NUM_EPOCHS = 10

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=34))
model.add(SpatialDropout1D(0.2))
model.add(BatchNormalization())

model.add(Conv1D(kernel_size=5, filters=75, activation='relu', dilation_rate=4))
model.add(BatchNormalization())
# model.add(Dropout(0.2))

model.add(Conv1D(kernel_size=3, filters=50, activation='relu', dilation_rate=2))
model.add(BatchNormalization())
# model.add(Dropout(0.2))

model.add(MaxPooling1D(pool_size=3))

# model.add(CuDNNLSTM(HIDDEN_LAYER_SIZE, return_sequences=True))
model.add(Bidirectional(CuDNNLSTM(HIDDEN_LAYER_SIZE)))

model.add(Dense(20))
model.add(Activation("sigmoid"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 34, 100)           2082700   
_________________________________________________________________
spatial_dropout1d_19 (Spatia (None, 34, 100)           0         
_________________________________________________________________
batch_normalization_v1_9 (Ba (None, 34, 100)           400       
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 18, 75)            37575     
_________________________________________________________________
batch_normalization_v1_10 (B (None, 18, 75)            300       
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 14, 50)            11300     
_________________________________________________________________
batch_normalization_v1_11 (B (None, 14, 50)            200       
__________

In [0]:
history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=20,validation_data=(X_test, Y_test))

Train on 491664 samples, validate on 49999 samples
Epoch 1/20
491664/491664 [==============================] - 10s 20us/sample - loss: 2.5575 - acc: 0.2525 - val_loss: 2.7330 - val_acc: 0.2242
Epoch 2/20
491664/491664 [==============================] - 8s 16us/sample - loss: 2.2795 - acc: 0.3287 - val_loss: 2.4474 - val_acc: 0.2972
Epoch 3/20
491664/491664 [==============================] - 8s 16us/sample - loss: 2.1935 - acc: 0.3495 - val_loss: 2.2817 - val_acc: 0.3322
Epoch 4/20
491664/491664 [==============================] - 8s 16us/sample - loss: 2.1348 - acc: 0.3642 - val_loss: 2.2985 - val_acc: 0.3334
Epoch 5/20
491664/491664 [==============================] - 8s 16us/sample - loss: 2.0859 - acc: 0.3759 - val_loss: 2.3252 - val_acc: 0.3308
Epoch 6/20
491664/491664 [==============================] - 8s 16us/sample - loss: 2.0422 - acc: 0.3872 - val_loss: 2.3521 - val_acc: 0.3294
Epoch 7/20
491664/491664 [==============================] - 8s 16us/sample - loss: 2.0012 - acc: 0.398

In [0]:
X_new = np.delete(X_train, np.s_[0:10], 1)
X_test_new = np.delete(X_test, np.s_[0:10], 1)

print(X_new.shape)
print(X_test_new.shape)

(491664, 24)
(49999, 24)


In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, CuDNNLSTM, SpatialDropout1D, Activation, Dropout, Bidirectional, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras import optimizers

vocab_size = 20827
EMBEDDING_SIZE = 500
HIDDEN_LAYER_SIZE = 40

NUM_EPOCHS = 10

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=24))
model.add(SpatialDropout1D(0.5))
# # model.add(BatchNormalization())

model.add(Conv1D(kernel_size=5, filters=100, activation='relu', dilation_rate=3))
# model.add(BatchNormalization())
model.add(Dropout(0.5))

# model.add(Conv1D(kernel_size=5, filters=25, activation='relu', dilation_rate=2))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

model.add(Bidirectional(CuDNNLSTM(HIDDEN_LAYER_SIZE)))

model.add(Dense(20))
model.add(Activation("sigmoid"))
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
rms_prop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss="categorical_crossentropy", optimizer=rms_prop, metrics=["accuracy"])
model.summary()
  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_37 (Embedding)     (None, 24, 500)           10413500  
_________________________________________________________________
spatial_dropout1d_31 (Spatia (None, 24, 500)           0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 12, 100)           250100    
_________________________________________________________________
batch_normalization_v1_22 (B (None, 12, 100)           400       
_________________________________________________________________
dropout_27 (Dropout)         (None, 12, 100)           0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 80)                45440     
_________________________________________________________________
dense_32 (Dense)             (None, 20)                1620      
__________

In [87]:
BATCH_SIZE = 8192*2
history = model.fit(X_new, Y_train, batch_size=BATCH_SIZE, epochs=10,validation_data=(X_test_new, Y_test))

Train on 491664 samples, validate on 49999 samples
Epoch 1/10
491664/491664 [==============================] - 16s 32us/sample - loss: 2.7260 - acc: 0.2116 - val_loss: 2.8602 - val_acc: 0.2170
Epoch 2/10
491664/491664 [==============================] - 11s 22us/sample - loss: 2.4973 - acc: 0.2598 - val_loss: 2.8194 - val_acc: 0.2519
Epoch 3/10
491664/491664 [==============================] - 11s 22us/sample - loss: 2.3738 - acc: 0.3072 - val_loss: 2.7214 - val_acc: 0.2402
Epoch 4/10
491664/491664 [==============================] - 11s 21us/sample - loss: 2.2925 - acc: 0.3292 - val_loss: 2.6382 - val_acc: 0.2253
Epoch 5/10
491664/491664 [==============================] - 10s 21us/sample - loss: 2.2375 - acc: 0.3409 - val_loss: 2.5516 - val_acc: 0.2557
Epoch 6/10
491664/491664 [==============================] - 10s 21us/sample - loss: 2.1980 - acc: 0.3489 - val_loss: 2.4767 - val_acc: 0.2867
Epoch 7/10
491664/491664 [==============================] - 10s 21us/sample - loss: 2.1671 - acc: